# Localize an ArcGIS Hub Site

### Import Python libraries

In [10]:
from arcgis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import FeatureLayerCollection

import json
import os

### Setup your connection to ArcGIS Online

In [11]:
gis = GIS(profile='joshdbs')
gis

GIS @ https://DBSNE.maps.arcgis.com version:7.1

In [12]:
# ago_working_directory = None
ago_working_directory = 'Localization Testing'

### What is the output locale code (output language)?

In [13]:
locale_codes = ['fr', 'es']

### Set the location to your field aliases file

In [14]:
field_aliases = None
with open(os.path.abspath('resources/sdgs/UNSD_STANDARD_FIELDS.json'), 'rb') as file:
    field_aliases = json.load(file)

field_aliases

{'en': {'REF_AREA': 'Reference Area',
  'OBS_VALUE': 'Observed Value',
  'INDICATOR': 'Indicator',
  'FREQ': 'Frequency',
  'SEX': 'Sex',
  'AGE': 'Age',
  'TIME_PERIOD': 'Time Period',
  'UNIT': 'Unit',
  'LOCATION': 'Location',
  'SOURCE_NAME': 'Source Name'},
 'fr': {'series_description': 'Description de la série',
  'series_code': 'Code de série',
  'series_release': 'Sortie de la série',
  'geoAreaCode': 'Code de zone géographique',
  'geoAreaName': 'Nom de la zone géographique',
  'REF_AREA': 'Zone de référence',
  'OBS_VALUE': 'Valeur observée',
  'INDICATOR': 'Indicateur',
  'FREQ': 'Fréquence',
  'Sex': 'Sexe',
  'Age': 'Âge',
  'TIME_PERIOD': 'Time Period',
  'Units': 'Unit',
  'LOCATION': 'Lieu',
  'SOURCE_NAME': 'Nom de la source'},
 'es': {'series_description': 'Serie Descripción',
  'series_code': 'Codigo de serie',
  'series_release': 'Lanzamiento de la serie',
  'geoAreaCode': 'Código de área geográfica',
  'geoAreaName': 'Nombre del area geografica',
  'REF_AREA': 'Are

### Create a mapping of existing Groups to new, localized Groups
For `to_id`, specify `None` if a new group is to be created

maybe use this mapping to specify any other properties that we want to put on the new, localized group

In [15]:
groups_config = [
    {
        'from_id': '1084de372b9b4d008ffa71473636438d',
        'locales': ['fr', 'es']
    }
]

In [16]:
for group_config in groups_config:
    
    locale_codes = group_config['locales']
    
    for locale_code in locale_codes:
        
        print ('STARTING LOCALE :: {}'.format(locale_code))       
        
        sdg_metadata = None
        with open('resources/sdgs/SDG_METADATA_API_{}.json'.format(locale_code), 'rb') as file:
            sdg_metadata = json.load(file)

        if sdg_metadata is None:
            print ('unable to get metadata file for locale :: {}'.format(locale_code))
            break

        icon_base_url = 'https://github.com/Esri/sdg-dash/raw/master/public/images/sdg/{}/TGG_Icon_Color_0{}.png'

        locale_fields = field_aliases[locale_code]
    
        origin_group = gis.groups.get(group_config['from_id'])

        # create the destination group
        new_group_name = '{} {}'.format(origin_group.title, locale_code)

        # TODO :: localize the Group Details
        new_group_dict = {
            "title": new_group_name,
            "tags": origin_group.tags,
            "description": "",
            "snippet": "",
            "access": "public",
            "isInvitationOnly": True,
            "isOpenData": True
        }

        print ('creating new Group "{}"'.format(new_group_name))
        dest_group = None
        try:
            dest_group = gis.groups.create_from_dict(new_group_dict)
        except Exception as ex:
            args = ex.args[0].split('\n')
            if args is not None and 'You already have a group named' in arg1:
                print ('Group :: {} :: already exists, getting its reference'.format(new_group_name))
                search_results = gis.groups.search('title:"{}"'.format(new_group_name))
                if len(search_results) > 0:
                    dest_group = search_results[0]
                else:
                    print ('Unable to Create or Get Group :: {} :: exiting ..'.format(new_group_name))
                    break

        # go through each layer in the origin_group and create a Hosted Feature Layer View in the new Group
        items = [item for item in origin_group.content() if item.type == 'Feature Service']
        for item in items:
            fl = item.layers[0]
            tags = item.tags

            sdg_ind_tag = None
            sdg_series_tag = None

            for tag in tags:
                if 'SDGIND' in tag:
                    sdg_ind_tag = tag
                elif 'SDGSERIES' in tag:
                    sdg_series_tag = tag

            if sdg_ind_tag is None or sdg_series_tag is None:
                break

            ind_parts = sdg_ind_tag.split('.')
            goal_id = ind_parts[1]
            target_id = '{}.{}'.format(goal_id, ind_parts[2])
            indicator_id = '{}.{}.{}'.format(goal_id, ind_parts[2], ind_parts[3])

            series_parts = sdg_series_tag.split('.')
            series_id = series_parts[1]
            print (goal_id,target_id, indicator_id, series_id)

            item_flc = FeatureLayerCollection.fromitem(item)

            temporary_view_name = '{}_{}'.format(tag.replace('.', '_').lower(), locale_code)
            print ('creating hosted fl view :: {} ..'.format(temporary_view_name))
            new_view = item_flc.manager.create_view(name=temporary_view_name)

            if ago_working_directory is not None:
                print ('moving new View to working directory :: {}'.format(ago_working_directory))
                new_view.move(ago_working_directory)

            view_flc = FeatureLayerCollection.fromitem(new_view)
            view_fl = view_flc.layers[0]

            # update the field alias
            fields = view_fl.properties.fields
            for field in fields:
                if field.name in locale_fields:
                    field.alias = locale_fields[field.name]

            view_fl.properties.editingInfo.lastEditDate = None

            print ('updating field aliases ..')
            view_fl.manager.update_definition(view_fl.properties)

            # update the AGO Item Card
            view_name = '{} - {}'.format(item.title, locale_code)
            snippet = item.title
            locale_tags = item.tags
            for series in sdg_metadata:
                if series['seriesCode'] == series_id:
                    view_name = 'Indicator {} - {}'.format(indicator_id, series['seriesDescription'])
                    snippet = view_name
                    locale_tags = item.tags + series['TAGS']
                    description = """
                        <b>Series {}</b>: {}<br/><br/>
                        <b>Indicator {}</b>: {}<br/><br/>
                        <b>Target {}</b>: {}<br/><br/>
                        <b>Goal {}</b>: {}<br/>
                        """.format(
                        series_id, series['seriesDescription'],
                        indicator_id, series['indicatorDescription'],
                        target_id, series['targetDescription'],
                        goal_id, series['goalDescription'])


            view_item_card = {
                "title": view_name,
                "description": description,
                "snippet": snippet,
                "tags": locale_tags,
                "categories": item.categories
            }

            thumbnail_url = icon_base_url.format(locale_code, goal_id)

            print ('updating Item card ..')
            new_view.update(item_properties=view_item_card, thumbnail=thumbnail_url)

            # share with everyone
            print ('sharing with Everyone & to destination Group :: {}'.format(dest_group.title))
            new_view.share(everyone=True,groups=dest_group.id)


print ('done')

STARTING LOCALE :: fr
creating new Group "SDG 1: No Poverty fr"
1 1.1 1.1.1 SI_POV_EMP1
creating hosted fl view :: sdgseries_si_pov_emp1_fr ..
moving new View to working directory :: Localization Testing
Folder not found for given owner
updating field aliases ..
updating Item card ..
sharing with Everyone & to destination Group :: SDG 1: No Poverty fr
STARTING LOCALE :: es
creating new Group "SDG 1: No Poverty es"
1 1.1 1.1.1 SI_POV_EMP1
creating hosted fl view :: sdgseries_si_pov_emp1_es ..
moving new View to working directory :: Localization Testing
Folder not found for given owner
updating field aliases ..
updating Item card ..
sharing with Everyone & to destination Group :: SDG 1: No Poverty es
done
